In [181]:
from pyspark.sql.types import StructType
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import math
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [182]:
my_conf = SparkConf()
my_conf.set("spark.app.name", "w12_Spark_1")
my_conf.set("spark.master", "local[*]")
my_conf.set("spark.driver.extraClassPath", "C:/Users/Vinoth/Downloads/sqljdbc_12.4.2.0_enu/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre8.jar")

spark = SparkSession.builder.config(conf = my_conf).getOrCreate()
spark


In [183]:
database_url = "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true"

database_name = "Learn_Sql"

jdbc_properties = {

"user": "Vinoth",

"password": "1234",

"driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"

}

In [184]:
# df_min_max = (spark.read
# .format("jdbc")
# .option("url", "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true;databaseName=Learn_Sql;")
# .option("query","select min(cost) as min_row, max(cost) as max_row from products")
# .option("user", "Vinoth")
# .option("password", "1234")
# .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
# .load().collect())

# min_value, max_value = df_min_max[0][0], df_min_max[0][1]

In [185]:
# df = (spark.read
#       .option("numPartitions", 3)
#       .option("PartitionColumn", "cost")
#       .option("lowerBound", int(min_value))
#       .option("upperBound", int(max_value))
#       .jdbc(url=f"{database_url};databaseName={database_name};", table="products", properties=jdbc_properties))

In [186]:
input_df = (spark.read
      .jdbc(url=f"{database_url};databaseName={database_name};", table="input", properties=jdbc_properties))


In [187]:
input_df.show()

+---+-------+-----+
| id|formula|value|
+---+-------+-----+
|  1|    1+4|   10|
|  2|    2+1|    5|
|  3|    3-2|   40|
|  4|    4-1|   20|
+---+-------+-----+



In [188]:
input_df2 = input_df.withColumn("operator_index_value", expr("instr(formula, '+') + instr(formula, '-')")) \
        .withColumn("left_value", expr("substring(formula, 1, operator_index_value-1)")) \
        .withColumn("right_value", expr("substring(formula, operator_index_value+1, len(formula))"))

In [189]:
input_df2.show()

+---+-------+-----+--------------------+----------+-----------+
| id|formula|value|operator_index_value|left_value|right_value|
+---+-------+-----+--------------------+----------+-----------+
|  1|    1+4|   10|                   2|         1|          4|
|  2|    2+1|    5|                   2|         2|          1|
|  3|    3-2|   40|                   2|         3|          2|
|  4|    4-1|   20|                   2|         4|          1|
+---+-------+-----+--------------------+----------+-----------+



In [190]:
input_df2.alias("t1").join(input_df2.alias("t2"), col("t1.left_value") == col("t2.id"), 'inner')\
.join(input_df2.alias("t3"), col("t1.right_value") == col("t3.id"), 'inner').select("t1.*", col("t2.value").alias("v1"), col("t3.value").alias("v2"))\
.select("id", "formula", "value", expr("v1+v2 as new_value")).orderBy("id").show()

+---+-------+-----+---------+
| id|formula|value|new_value|
+---+-------+-----+---------+
|  1|    1+4|   10|       30|
|  2|    2+1|    5|       15|
|  3|    3-2|   40|       45|
|  4|    4-1|   20|       30|
+---+-------+-----+---------+

